In [1]:
path='/FileStore/tables/MLLib/train_for_MLlib.csv'


data=sqlContext.read.format("csv")\
        .option("header","true")\
        .option("inferSchmea","true")\
        .load(path)

display(data)

In [2]:
# We use labeled point mostly for classification purposes which will essentially convert into labels and features

from pyspark.mllib.regression import LabeledPoint

dataDf=data.select("HistCTR","Position","IsClick")\
      .map(lambda r: LabeledPoint(r[2],[r[0],r[1]]))\
      .toDF()
    
print(dataDf.head(5))
    

In [3]:
(trainingData, testData) = dataDf.randomSplit([0.7, 0.3], seed = 100)

from pyspark.ml.classification import LogisticRegression

# Create initial LogisticRegression model
lr = LogisticRegression(labelCol="label", featuresCol="features", maxIter=10)

# Train model with Training Data
lrModel = lr.fit(trainingData)



In [4]:
predictionsA=lrModel.transform(testData)
print(predictionsA)

In [5]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Evaluate model
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
evaluator.evaluate(predictionsA)

In [6]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, [0.01, 0.5, 2.0])
             .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])
             .addGrid(lr.maxIter, [1, 5, 10])
             .build())

In [7]:

cv = CrossValidator(estimator=lr, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=3)

cvModel = cv.fit(testData)


In [8]:
predictions = cvModel.transform(testData)
evaluator.evaluate(predictions)